In [ ]:
import time
import ccxt
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

from bs4 import BeautifulSoup
import re
import numpy as np
import os

In [ ]:
## 导入数据
file_path = r'C:\Users\Admin\Desktop\backtrader\datas\binance_BTCUSDT.csv'
file_path = r'C:\Users\Admin\Desktop\backtrader\datas\binance_BTCUSDT.csv'
file_path = '../../datas/binance_BTCUSDT.csv'
funding_data = pd.read_csv(file_path)
file_path = '../../datas/币安USDT资费.csv'
borrow_data = pd.read_csv(file_path)
borrow_data = borrow_data.drop_duplicates(subset='时间')
# 将时间列转换为时间戳
funding_data['datetime'] = pd.to_datetime(funding_data['datetime']).astype('int64') // 10**9
borrow_data['时间'] = pd.to_datetime(borrow_data['时间']).astype('int64') // 10**9

funding_data = funding_data.sort_values(by='datetime', ascending=True).reset_index(drop=True)
borrow_data = borrow_data.sort_values(by='时间', ascending=True).reset_index(drop=True)

# 统一时间间隔为8小时
# 先找出最早和最晚的时间
min_time = borrow_data['时间'].min()
max_time = borrow_data['时间'].max()




In [166]:
## 导入数据
file_path = r'C:\Users\Admin\Desktop\backtrader\datas\binance_BTCUSDT.csv'
file_path = r'C:\Users\Admin\Desktop\backtrader\datas\binance_BTCUSDT.csv'
file_path = '../../datas/binance_BTCUSDT.csv'
funding_data = pd.read_csv(file_path)
file_path = '../../datas/币安USDT资费.csv'
borrow_data = pd.read_csv(file_path)
borrow_data = borrow_data.drop_duplicates(subset='时间')
# 将时间列转换为时间戳
funding_data['datetime'] = pd.to_datetime(funding_data['datetime']).astype('int64') // 10**9
borrow_data['时间'] = pd.to_datetime(borrow_data['时间']).astype('int64') // 10**9

funding_data = funding_data.sort_values(by='datetime', ascending=True).reset_index(drop=True)
borrow_data = borrow_data.sort_values(by='时间', ascending=True).reset_index(drop=True)

# 统一时间间隔为8小时
# 先找出最早和最晚的时间
min_time = max(borrow_data['时间'].min(), funding_data['datetime'].min())
max_time = min(borrow_data['时间'].max(), funding_data['datetime'].max())

# 生成8小时间隔的时间序列
time_series = pd.date_range(start=pd.Timestamp.fromtimestamp(min_time), end=pd.Timestamp.fromtimestamp(max_time), freq='8H').astype('int64') // 10**9

# 对funding_data进行重采样
funding_data.set_index('datetime', inplace=True)
funding_data = funding_data.reindex(time_series).reset_index()

# 对borrow_data进行重采样
borrow_data.set_index('时间', inplace=True)
borrow_data = borrow_data.reindex(time_series).reset_index()

# 合并两个DataFrame
merged_data = pd.concat([funding_data, borrow_data], axis=1)

columns_to_fill = ['年利率', '日利率', '小时利率'] # 简单前向填充，再删除缺失数据
merged_data[columns_to_fill] = merged_data[columns_to_fill].fillna(method='ffill', limit=2)
merged_data = merged_data.dropna()
merged_data = merged_data.drop('Unnamed: 0', axis=1)

# 去除列名重复的列（保留第一个出现的列名）
merged_data = merged_data.loc[:, ~merged_data.columns.duplicated(keep='first')]
print(merged_data)

           index  fundingRate     年利率     日利率      小时利率
2     1690963200     0.000100  0.0486  0.0001  0.000006
3     1690992000     0.000100  0.0486  0.0001  0.000006
4     1691020800     0.000100  0.0486  0.0001  0.000006
5     1691049600     0.000100  0.0510  0.0001  0.000006
6     1691078400     0.000100  0.0510  0.0001  0.000006
...          ...          ...     ...     ...       ...
1839  1743868800     0.000004  0.0582  0.0002  0.000007
1840  1743897600     0.000002  0.0582  0.0002  0.000007
1841  1743926400     0.000054  0.0555  0.0002  0.000006
1842  1743955200    -0.000006  0.0555  0.0002  0.000006
1843  1743984000     0.000049  0.0555  0.0002  0.000006

[1842 rows x 5 columns]


In [ ]:
def fill_missing_with_mean(df, col):
    """
    改进版的三等分点均值填充函数
    逻辑：在每两个非缺失值之间，计算三等分点并用两端均值填充
    """
    # 获取非缺失值的索引
    non_missing = df[col].dropna()
    if len(non_missing) < 2:
        return df  # 至少需要两个点才能计算
    
    # 转换为数值索引（如果索引不是数字）
    df = df.reset_index(drop=True) if not df.index.is_numeric() else df
    
    for i in range(len(non_missing) - 1):
        start_idx = non_missing.index[i]
        end_idx = non_missing.index[i+1]
        
        # 计算间隔和填充点
        interval = end_idx - start_idx
        if interval <= 1:  # 相邻点不需要填充
            continue
            
        # 计算三等分点位置
        fill_positions = [
            start_idx + (interval // 3),
            start_idx + 2 * (interval // 3)
        ]
        
        # 计算两端均值
        mean_val = (df.at[start_idx, col] + df.at[end_idx, col]) / 2
        
        # 填充有效位置
        for pos in fill_positions:
            if pd.isna(df.at[pos, col]):  # 只有确实缺失才填充
                df.at[pos, col] = mean_val
                
    return df

# 使用示例
columns_to_fill = ['年利率', '日利率', '小时利率']
for col in columns_to_fill:
    if col in merged_data.columns:
        merged_data = fill_missing_with_mean(merged_data.copy(), col)  # 使用copy避免链式赋值警告
